In [ ]:
pip install transformers PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline
import PyPDF2

In [ ]:

# Initialize the Hugging Face summarizer model
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text

In [ ]:
# Function to highlight specific key clauses and format the output with line breaks
def highlight_clauses(text):
    # Define key terms to highlight
    key_terms = ['Confidential Information', 'Services', 'Fees', 'Termination', 'Dispute Resolution', 'Arbitration', 'Amendments', 'Obligations']

    # Split the text into individual sentences based on periods
    sentences = text.split('. ')

    highlighted_text = ""

    # Highlight clauses by bolding key terms
    for sentence in sentences:
        for term in key_terms:
            if term in sentence:
                sentence = sentence.replace(term, f"**{term}**")  # Bold the key terms
        highlighted_text += sentence + ".\n\n"  # Add new line after each sentence

    return highlighted_text

In [ ]:
# Main function to summarize the document and highlight key clauses
def legal_document_summarizer(pdf_path):
    # Extract text from the PDF
    extracted_text = extract_text_from_pdf(pdf_path)

    # Split the extracted text into smaller chunks
    chunk_size = 512 # Adjust this value as needed
    chunks = [extracted_text[i:i + chunk_size] for i in range(0, len(extracted_text), chunk_size)]

    summarized_text = ""
    # Summarize each chunk individually
    for chunk in chunks:
        summary = summarizer(chunk, max_length=150, min_length=30, do_sample=False)
        summarized_text += summary[0]['summary_text'] + " "

    # Highlight key clauses in the summarized text
    formatted_summary = highlight_clauses(summarized_text)

    return formatted_summary

In [ ]:
pdf_path = "/content/drive/My Drive/LEGAL DOCUMENT/LegalDoc (4).pdf"
summary = legal_document_summarizer(pdf_path)

Your max_length is set to 150, but your input_length is only 148. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)
Your max_length is set to 150, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)
Your max_length is set to 150, but your input_length is only 113. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 150, but your input_length is only 101. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [ ]:
# Print the formatted summary with highlighted clauses
print("Summarized Document with Highlighted Clauses:\n")
print(summary)

Summarized Document with Highlighted Clauses:

April 9, 2025.

Sample Legal Agreement.

Party 1 agrees to provide the following **Services** to Party 2:.

Service 1: Description of the service.

Service 2: description of another service.

Party 1 agrees to maintain the confidentiality of any Confidential In-formation shared during the term of this Agreement.

Party 2 shall pay Party 1 the agreed-upon **Fees** for**Services**, as outlined in the Agreement.

This shall include access to confidential company records, systems, and employees.

Payment terms are net 30 days from invoice date.

Failure to pay the **Fees** within the agreed terms will result in late fees and possible **Termination** of the Agreement.

In case of a **Termination** , both parties shall promptly return all property, including any **Confidential Information**, to the disclosing party.

If the dispute cannot be resolved through negotiation, the parties agree to submit the dispute tomediation in accordance with the 